In [ ]:
import os
import json
import glob
import torch
from sklearn.model_selection import train_test_split
import logging


In [2]:
# Logs if there are errors in the conversion process
logging.basicConfig(filename="conversion_errors.log", level=logging.ERROR)

In [4]:
# Use of computer configured gpu
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

Using device: mps


In [5]:
#Step 1: Load `meta.json` and Extract Class Mapping
# Load class names from meta.json
META_JSON_FILE = "dataset/DatasetNinja_RDD2022_MIX/meta.json"
with open(META_JSON_FILE, "r") as meta_file:
    meta_data = json.load(meta_file)

class_map = {cls["title"] : idx for idx, cls in enumerate(meta_data["classes"])}
print("Loaded class mapping:", class_map)

Loaded class mapping: {'alligator crack': 0, 'block crack': 1, 'longitudinal crack': 2, 'other corruption': 3, 'pothole': 4, 'repair': 5, 'transverse crack': 6}


In [6]:
# Step 2: Create id for all class
class_map = {
    "longitudinal crack" : "D00",
    "transverse crack" : "D10",
    "alligator crack": "D20",
    "block crack": "D30",
    "pothole": "D40",
    "manhole cover": "D50",
    "other corruption": "D60"
}
class_id_map = {v: k for k, v in enumerate(class_map.values())}
print(class_id_map)

{'D00': 0, 'D10': 1, 'D20': 2, 'D30': 3, 'D40': 4, 'D50': 5, 'D60': 6}


In [13]:
# Step 3: Process All JSON Files
INPUT_JSON_DIR_TRAIN = "datasets/yolo_dataset/train/annotations/"
INPUT_JSON_DIR_VAL = "datasets/yolo_dataset/val/annotations/"

OUTPUT_DIR_TRAIN = "datasets/yolo_dataset/train/labels/"
OUTPUT_DIR_VAL = "datasets/yolo_dataset/val/labels/"

json_files_train = glob.glob(os.path.join(INPUT_JSON_DIR_TRAIN, "*.json"))
json_files_test = glob.glob(os.path.join(INPUT_JSON_DIR_VAL, "*.json"))


In [9]:
def convert_json_to_yolo(json_file, output_dir):
    with open(json_file, 'r') as json_f:
        data = json.load(json_f)
    
    image_width, image_height = data["size"]["width"], data["size"]["height"]
    yolo_annotations = []
   
 
    for obj in data["objects"]:
        class_name = obj["classTitle"]
        if class_name not in class_map:
            logging.error(f"Class '{class_name}' not found in class mapping. Skipping object.")
            continue

        class_id = class_id_map[class_map.get(class_name)]
       
        points = obj["points"]["exterior"]
        x_min, y_min = points[0]
        x_max, y_max = points[1]
        
        x_center = (x_min + x_max) / 2.0 / image_width
        y_center = (y_min + y_max) / 2.0 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

       
        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    output_txt = os.path.join(output_dir, os.path.basename(json_file).replace(".jpg.json", ".txt"))
    os.makedirs(output_dir, exist_ok=True)
    with open(output_txt, "w") as f:
        f.write("\n".join(yolo_annotations))

    return f"Converted: {json_file} → {output_txt}"

In [ ]:
# Converting train folder annotations
print("--------------------------------")
print("Processing Train Annotations...")
for json_file in json_files_train:
    convert_json_to_yolo(json_file, OUTPUT_DIR_TRAIN)
print("--------------------------------")
print("Processed Train Annotations...")

--------------------------------
Processing Train Annotations...
--------------------------------
Processed Train Annotations...


In [14]:
# Converting Val folder annotations
print("--------------------------------")
print("Processing Validation Annotations...")
for json_file in json_files_test:
    convert_json_to_yolo(json_file, OUTPUT_DIR_VAL)
print("--------------------------------")
print("Processed Validation Annotations...")

--------------------------------
Processing Validation Annotations...
--------------------------------
Processed Validation Annotations...
